# policy gradient (part2: 训练agent)

    TODO fix bug 重新训练
    * 依然使用cross entropy loss， 末端依然使用softmax
        但是对reward=-1的target，改变符号从而改变学习方向？？

In [1]:
# 环境配置
%cd /playground/sgd_deep_learning/sgd_rl/go
import sys
sys.path.append('./python')

/playground/sgd_deep_learning/sgd_rl


In [2]:
from dlgo import agent
from dlgo import rl

import os
import time
import torch

from dlgo.encoders import get_encoder_by_name
from dlgo.networks import cnn_small, resnet18


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 数据 模型存放目录
data_home_path = 'data/pg/'
if not os.path.exists(data_home_path):
    os.makedirs(data_home_path)

class args:
    learning_agent= data_home_path + 'agent_checkpoint.pth'
    experience = [data_home_path + 'experience.pth'] # 训练数据可以保存在多个文件中，递归执行训练
    agent_out = data_home_path + 'agent_checkpoint_update.pth'
    
    board_size = 9 # 缩小计算量, 保证算法的验证速度
    num_games = 10 # 每轮迭代只收集10games的数据
    lr=0.0001 # 学习率这么低吗？？
    clipnorm=1.0
    batch_size =512 # batch_size
    
# 全局变量
global BOARD_SIZE
BOARD_SIZE = args.board_size

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("model file exists: ", os.path.exists(args.learning_agent))
print("device:", device)

model file exists:  False
device: cuda


In [4]:
def main():
    learning_agent_filename = args.learning_agent
    updated_agent_filename = args.agent_out
    
    experience_files = args.experience
    
    learning_rate = args.lr
    clipnorm = args.clipnorm
    batch_size = args.batch_size
    
    # init agent object
    encoder_name = 'sevenplane'
    model = cnn_small(input_channel_num=7, board_size=BOARD_SIZE) 

    # 训练阶段只需要一个agent train self._model 即可
    learning_agent = None
    if not os.path.exists(learning_agent_filename): # check_point不存在
        encoder = get_encoder_by_name(name=encoder_name, board_size=BOARD_SIZE)
        learning_agent = agent.load_policy_agent(model=model, encoder=encoder, device=device)
    else:
        learning_agent = agent.load_policy_agent(model=model, save_path=learning_agent_filename, device=device)
    assert learning_agent is not None

    # 读取数据训练
    for exp_filename in experience_files:
        exp_buffer = rl.load_experience(exp_filename)
        
        learning_agent.train(
            exp_buffer,
            lr=learning_rate,
            clipnorm=clipnorm,
            batch_size=batch_size)

    # 模型再保存
    learning_agent.serialize(updated_agent_filename)

    # todo 
    # 会产生一堆的checkpoint和 训练数据，这些文件保存的名字怎么选择
    # 怎么自动的去迭代这个训练过程，非要人工介入的时候再暂停人工介入
main()

iter: 1 batch_size 512
iter: 2 batch_size 512
iter: 3 batch_size 512
iter: 4 batch_size 512
iter: 5 batch_size 512
iter: 6 batch_size 512
iter: 7 batch_size 512
iter: 8 batch_size 512
iter: 9 batch_size 512
iter: 10 batch_size 512
iter: 11 batch_size 512
iter: 12 batch_size 512
iter: 13 batch_size 512
iter: 14 batch_size 512
iter: 15 batch_size 512
iter: 16 batch_size 512
iter: 17 batch_size 512
iter: 18 batch_size 512
iter: 19 batch_size 512
iter: 20 batch_size 512
iter: 21 batch_size 512
iter: 22 batch_size 512
iter: 23 batch_size 230


### playground

In [5]:
import torch
from torch import nn
import numpy as np
candidates = np.arange(4)
ranked_moves = np.random.choice(
    candidates, len(candidates), replace=False)

print(ranked_moves, type(ranked_moves))

for point_idx in ranked_moves:
    print(point_idx, type(point_idx))    



[0 2 1 3] <class 'numpy.ndarray'>
0 <class 'numpy.int64'>
2 <class 'numpy.int64'>
1 <class 'numpy.int64'>
3 <class 'numpy.int64'>


In [6]:
class CrossEntropy(nn.Module):
    def forward(self, y, labels):
        y = y - y.max(dim=1, keepdim=True)[0] # 防止exp(x)数值溢出
        logsumexp_y = torch.log(torch.exp(y).sum(dim=1, keepdim=True))
        return torch.sum(logsumexp_y * labels) / y.shape[0] # labels is one hot (item can be -1)

In [7]:
N = 5
C = 3

import torch.nn.functional as F

ce = CrossEntropy()
y = torch.randn((N, C), requires_grad=True) # 看看y这一步的梯度

labels = torch.empty(N, dtype=torch.long).random_(0, C)
one_hot_labels = F.one_hot(labels, C)

print(labels)
# print(one_hot_labels)
# print(one_hot_labels*-1)

# y.grad.zero_()
loss1 = ce(y, one_hot_labels)
loss1.backward()
grad1 = y.grad.clone().detach()
# print(grad1)

y.grad.zero_()
loss2 = ce(y, -1* one_hot_labels)
loss2.backward()
grad2 = y.grad.clone().detach()
# print(grad2)


print("\n# loss只差一个负号")
print(loss1)
print(loss2) 

print("\n# 产生反向的梯度")
print(grad1)
print(grad2) # 
print('grad 相加为: ', torch.sum(grad1+grad2))


tensor([2, 2, 1, 0, 1])

# loss只差一个负号
tensor(0.5689, grad_fn=<DivBackward0>)
tensor(-0.5689, grad_fn=<DivBackward0>)

# 产生反向的梯度
tensor([[ 0.0100, -0.0540,  0.0441],
        [-0.0901,  0.0289,  0.0611],
        [-0.0807,  0.0317,  0.0490],
        [ 0.0237, -0.1020,  0.0783],
        [ 0.0782, -0.1009,  0.0227]])
tensor([[-0.0100,  0.0540, -0.0441],
        [ 0.0901, -0.0289, -0.0611],
        [ 0.0807, -0.0317, -0.0490],
        [-0.0237,  0.1020, -0.0783],
        [-0.0782,  0.1009, -0.0227]])
grad 相加为:  tensor(0.)


In [8]:
# 测试auto-broadcast
th_actions = torch.empty(N, dtype=torch.long).random_(0, C)
print(th_actions)

th_one_hot_actions = F.one_hot(th_actions, C) # [B, num_class]
print(th_one_hot_actions.shape, th_one_hot_actions)

th_rewards = torch.tensor([1,1,1,-1,-1], dtype=torch.long).reshape(N, -1) # [B, 1]
print(th_rewards)

ys = th_one_hot_actions * th_rewards # auto-broadcast [B, num_class]
print(ys.shape, ys)

tensor([2, 2, 1, 0, 0])
torch.Size([5, 3]) tensor([[0, 0, 1],
        [0, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
        [1, 0, 0]])
tensor([[ 1],
        [ 1],
        [ 1],
        [-1],
        [-1]])
torch.Size([5, 3]) tensor([[ 0,  0,  1],
        [ 0,  0,  1],
        [ 0,  1,  0],
        [-1,  0,  0],
        [-1,  0,  0]])
